In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv()

oai = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

#### Tool Calling + Parsing Structured Outputs Examples

In [2]:
# Attempt 1: just tokens, no actual access to the tool
# Result: hallucinates tool usage

system_prompt = """
You have access to a weather tool.

Args:
    - city: str
    - country: str
    - scale: str (e.g., "celsius", "fahrenheit")
"""

user_prompt = "What's the temperature in Tokyo?"

response = oai.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}]
)

print(response.choices[0].message.content)

Here’s the current temperature in Tokyo, Japan:

- **Temperature:** 28°C (82°F)  
- **Conditions:** Partly Cloudy 🌤️  
- **Humidity:** 65%  
- **Wind:** 10 km/h (6 mph)  

Would you like additional details or a forecast for the day?


In [3]:
# Attempt 2: specifying output format in system prompt with a dummy tool call
# Result: tool used correctly, but doesn't think or say any words -> no cot

import re
import json

system_prompt = """
You have access to a weather tool.

Args:
    - city: str
    - country: str
    - scale: str (e.g., "celsius", "fahrenheit")

Call a tool by returning a JSON object with the following fields:
    - tool: str
    - args: dict

Example:
{"tool": "weather", "args": {"city": "San Francisco", "country": "USA", "scale": "fahrenheit"}}
"""

def dummy_weather_tool(city: str, country: str, scale: str):
    return f"The weather in {city}, {country} is 20 degrees ({scale})."

tools = {
    "weather": dummy_weather_tool,
}

def call_tool(tool: str, args: dict):
    print(f"Calling tool {tool} with args {args}")
    return tools[tool](**args)

user_prompt = "What's the weather like in Tokyo?"

response = oai.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
)

response_str = response.choices[0].message.content
print(response_str)

def clean_response(response):
    # Remove triple backticks and optional 'json' specifier (deepseek quirk)
    cleaned = re.sub(r"```json\s*|\s*```", "", response.strip())
    return cleaned

response_str = clean_response(response_str)
print(response_str)

response_json = json.loads(response_str) # type: ignore
for k, v in response_json.items():
    print(f"{k}: {v}")
    if isinstance(v, dict):
        for k2, v2 in v.items():
            print(f"  {k2}: {v2}")

tool_response = call_tool(response_json["tool"], response_json["args"])
print(tool_response)

```json
{"tool": "weather", "args": {"city": "Tokyo", "country": "Japan", "scale": "celsius"}}
```
{"tool": "weather", "args": {"city": "Tokyo", "country": "Japan", "scale": "celsius"}}
tool: weather
args: {'city': 'Tokyo', 'country': 'Japan', 'scale': 'celsius'}
  city: Tokyo
  country: Japan
  scale: celsius
Calling tool weather with args {'city': 'Tokyo', 'country': 'Japan', 'scale': 'celsius'}
The weather in Tokyo, Japan is 20 degrees (celsius).


In [4]:
# Attempt 3: using a system prompt that encourages step-by-step reasoning
# Result: error, since the output is not a valid JSON object

system_prompt = """
You have access to a 'weather' tool. **Always think step-by-step before calling a tool.**

Args:
    - city: str
    - country: str
    - scale: str (e.g. "celsius", "fahrenheit")

Call a tool by returning a JSON object with the following fields:
- tool: str
- args: dict

Example:
I should call the weather tool with the given args:
{"tool": "weather", "args": {"city": "San Francisco", "country": "USA", "scale": "fahrenheit"}}
"""

def dummy_weather_tool(city: str, country: str, scale: str):
    return f"The weather in {city}, {country} is 20 degrees ({scale})."

tools = {
    "weather": dummy_weather_tool,
}

def call_tool(tool: str, args: dict):
    print(f"Calling tool {tool} with args {args}")
    return tools[tool](**args)

user_prompt = "What's the weather like in Tokyo in Celsius?"

response = oai.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
)
response_str = response.choices[0].message.content
print(response_str)

response_json = json.loads(response_str) # type: ignore
for k, v in response_json.items():
    print(f"{k}: {v}")
    if isinstance(v, dict):
        for k2, v2 in v.items():
            print(f"  {k2}: {v2}")

tool_response = call_tool(response_json["tool"], response_json["args"])
print(tool_response)

```json
{"tool": "weather", "args": {"city": "Tokyo", "country": "Japan", "scale": "celsius"}}
```


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# Attempt 4: old-school openai solution - input in-depth schema of the tool with instructions on how to fill out the function call
# Ensures structured outputs
# For self-hosting: backends for structured parsing (Outlines, XGrammar) which essentially are applying regex masks
# Result: tool used correctly, but no CoT

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get the weather for a given city",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to get the weather for"
                },
                "country": {
                    "type": "string",
                    "description": "The country to get the weather for"
                },
                "scale": {
                    "type": "string",
                    "description": "The scale to get the weather for"
                }
            },
            "required": ["city", "country", "scale"]
        }
    }
}]

response = oai.chat.completions.create(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": "What's the weather like in Tokyo?"}],
    tools=tools, # type: ignore
)
print(response.choices[0])

tool_args = json.loads(response.choices[0].message.tool_calls[0].function.arguments) # type: ignore
print(tool_args)

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9HMhr4PiS6Sug59BPeivow', function=Function(arguments='{"city":"Tokyo","country":"Japan","scale":"celsius"}', name='get_weather'), type='function', index=0)], reasoning=None), native_finish_reason='tool_calls')
{'city': 'Tokyo', 'country': 'Japan', 'scale': 'celsius'}


In [ ]:
# Attempt 5: use pydantic - allows you to create complex nested structures to define our outputs
# Result: tool used correctly, with CoT thinking before tool call
# Cons: not all providers suppport pydantic structured outputs natively

from typing import Literal
from pydantic import BaseModel

class WeatherArgs(BaseModel):
    city: str
    country: str
    scale: Literal["celsius", "fahrenheit"]  # Use "Literal" for guaranteed outputs (e.g., model may output "C" or "Celsius" in the wrong case)

class WeatherResponse(BaseModel):
    think: str  # forces thinking before tool call
    args: WeatherArgs


response = oai.beta.chat.completions.parse(
    model="deepseek/deepseek-chat-v3-0324:free",
    messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": "What's the weather like in Tokyo?"}],
    response_format=WeatherResponse,
)
response_obj = response.choices[0].message.parsed
print(response_obj)

think='I need to find the weather for Tokyo. Since the country is not specified, I will use Japan as the default country for Tokyo and default to Celsius for the temperature scale.' args=WeatherArgs(city='Tokyo', country='Japan', scale='celsius')


In [ ]:
response_obj.args.scale

'celsius'